In [ ]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import re
from datetime import datetime

In [ ]:
#입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        page_num1 = num
        return page_num1
    elif num == 0:
        page_num2 = num+1
        return page_num2
    else:
        page_num3 = num+9*(num-1)
        return page_num3

In [ ]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search,start_pg,end_pg,start_date, end_date):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + f'&sm=tab_opt&nso=so:r,p:from{start_date}to{end_date}'+"&start=" + str(start_page)
        print("생성url: ",url)
        return url
    else:
        urls= []
        for i in range(start_pg, end_pg+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + f'&sm=tab_opt&nso=so:r,p:from{start_date}to{end_date}' + "&start=" + str(page)
            urls.append(url)
        print("생성url: ",urls)
        return urls

In [ ]:
def get_links (search_urls) :
  headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

  naver_urls = []
  press_name = []


  cnt = 0
  for i in search_urls :
      cnt =+ 1
      print("-----------> ", cnt)
      print("search_urls: ", i)
      response = requests.get(i, headers=headers)
      html = response.text
      soup = BeautifulSoup(html, "html.parser")
  
      news = soup.select("div > div.news_info > div.info_group > a ")     # a:nth-of-type(3)
      article = soup.select("a.info.press") # i당 10개
      articles = []
      for art in article:
        articles.append(art)

      #print("\n뉴스 : ",news)
      #print("\n아티클 : ", article)
      cnt = 0
      idx = 0
      
      for n in news:
        cnt += 1
        print("-- ", cnt, " : ", n)

        if "sid1=106" in n['href'] :
              pass
        elif "sports" in n['href'] :
              pass
            
        elif  n['href'].startswith("https://news.naver.com"):
              print("\n네이버뉴스 입니다")
              news_url = n['href']
              naver_urls.append(news_url)
              try:
                press_name.append(articles[idx-1].text.replace("선정",""))
              except:
                print('none')
              
              
        else:
          idx += 1
          
  return naver_urls, press_name

In [ ]:
def get_content (naver_urls) : 
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}


    titles = []
    contents = []
    news_date = []
    report = []

    for i in naver_urls :
        original_html = requests.get (i, headers=headers)
        h = original_html.content
        html = BeautifulSoup(h, "html.parser")
        title = html.select("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        title = ''.join(str(title)) # 문자열 합치기
        
        del_str = '<[^>]*>'
        title = re.sub(pattern=del_str, repl='',string=title)
        titles.append(title) 
        
        # 본문
        content = html.select("#dic_area")
        content = ''.join(str(content))
        content = re.sub(pattern=del_str, repl='',string=content)
        del_str2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(del_str2,'')
        # print(contents)
        contents.append(content)
        
        # 기사작성일
        p_date = html.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")
        4
        
        if  "sid1=106" in i:
            sports = html.select("#content > div.end_ct > div > div.article_info > span > em")
            pattern1=r'([0-9]{4})-([0-9]{2})-([0-9]{2})'
            regex_result1 = re.search(pattern1, str(sports))
            pub_date1 = regex_result1.group(0)
            news_date.append(pub_date1)
                    
        else : 
            pattern=r'([0-9]{4})-([0-9]{2})-([0-9]{2})'
            regex_result = re.search(pattern, str(p_date))
            pub_date = regex_result.group(0)
            news_date.append(pub_date)
    
        # 기자이름
        
        r = html.select("#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_journalist > a > em")
        if len(r) == 0:
            r = html.select("#contents > div.byline > p > span")
        report.append(r)
    
    reporters = []

    for a in report :
        if len(a) ==0 :
            reporters.append(r)
        
        else :
            b = a[0].text # 위치값 
            reporters.append(b.replace(' 기자',''))

    return titles, contents, news_date, reporters

In [ ]:
def makeFrame (news_date, press_name, reporters, naver_urls, titles, contents):
    data = pd.DataFrame({"Pub_date" : news_date, "Press_name" : press_name, "Reporter" : reporters, "News_url" : naver_urls, "Title" : titles, "Content" : contents})
    return data

In [ ]:
#검색어 입력
search = input("검색할 키워드를 입력해주세요:")

#검색시작일
start_date = input("검색 시작 날짜 (YYYYmmdd) : ")

#검색종료일
end_date = input("검색 종료 날짜 (YYYYmmdd) : ")

#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 종료 페이지: ",page2,"페이지")   

# naver url 생성
search_urls = makeUrl(search,page, page2,start_date, end_date)
naver_urls, press_name = get_links (search_urls)
titles, contents, news_date, reporters = get_content (naver_urls)
data = makeFrame (news_date, press_name, reporters, naver_urls, titles, contents)

In [ ]:
data

In [ ]:
data["Title"] = data["Title"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
data["Title"] = data["Title"].str.replace("[0-9]+", repl=r" ", regex=True)
data["Title"] = data["Title"].str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣]',repl=r" ",regex=True)

data["Content"] = data["Content"].str.replace(pat=r'(\[a-zA-Z0-9\_.+-\]+@\[a-zA-Z0-9-\]+.\[a-zA-Z0-9-.\]+)', repl=r'', regex=True)
data["Content"] = data["Content"].str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣]',repl=r" ",regex=True)
data["Content"] = data["Content"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
data["Content"] = data["Content"].str.replace(pat=r'[\r|\n]', repl=r' ', regex=True)
data["Content"] = data["Content"].str.replace(pat=r'<[^>]*>', repl=r' ', regex=True)
data["Content"] = data["Content"].str.replace("[0-9]+", repl=r" ", regex=True)

data['Reporter'] = data['Reporter'].str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣]',repl=r" ",regex=True)

In [ ]:
data.isnull().sum()

In [ ]:
data = data.fillna("기타")

# 중복 확인 후 제거
final_df = data.copy()
dup = final_df.duplicated(['News_url']).sum()
print(dup)
final_df=final_df.drop_duplicates(['News_url'])

In [ ]:
final_df

In [ ]:
# csv 저장
import datetime

now = datetime.datetime.now()
filename = "{}".format(search) + "_" + "{}".format(end_date)
df = final_df.to_csv(filename +'.csv', encoding = "utf-8-sig", index = False)